<a href="https://colab.research.google.com/github/Bhuvaneswaran-k/cointab/blob/main/Cointab_assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
! git clone https://github.com/Bhuvaneswaran-k/cointab
ord_report=pd.read_excel("/content/cointab/Company X - Order Report.xlsx")
pin_code=pd.read_excel("/content/cointab/Company X - Pincode Zones.xlsx")
sku_master=pd.read_excel("/content/cointab/Company X - SKU Master.xlsx")
invoice=pd.read_excel("/content/cointab/Courier Company - Invoice.xlsx")
rate=pd.read_excel("/content/cointab/Courier Company - Rates.xlsx")

df=pd.DataFrame()
df["Order Id"]= invoice["Order ID"]
df["AWB Number"]=invoice["AWB Code"]

merge = pd.merge(ord_report,sku_master,on="SKU")
merge["Total weight as per X (KG)"]=((merge['Order Qty']*merge["Weight (g)"])/1000)
merge.rename(columns = {'ExternOrderNo':'Order Id'}, inplace = True)

pivot_df=pd.pivot_table(merge,index="Order Id",aggfunc="sum")
pivot_df.reset_index(inplace=True)

df=pd.merge(df,pivot_df,on="Order Id")

def assign_weight_slab(x):
    if x <= 0.5:
        return 0.5
    elif x < 1.0:
        return 1.0
    elif x < 1.5:
        return 1.5
    elif x < 2.0:
        return 2.0
    elif x < 2.5:
      return 2.5
    elif x < 3:
      return 3
    elif x < 3.5:
      return 3.5
    elif x < 4:
      return 4
    elif x < 4.5:
      return 4.5      
    else:
        return 5  
df['Weight slab as per X (KG)'] = df['Total weight as per X (KG)'].apply(assign_weight_slab)
df["Total weight as per Courier Company (KG)"]=invoice["Charged Weight"]
df["Weight slab charged by Courier Company (KG)"]=df["Total weight as per Courier Company (KG)"].apply(assign_weight_slab)

for_pin=invoice.drop(["AWB Code","Type of Shipment","Warehouse Pincode","Charged Weight","Billing Amount (Rs.)",'Type of Shipment',"Zone"],axis=1)
merge2=pd.merge(pin_code,for_pin,on="Customer Pincode",)
pin_1=merge2.drop_duplicates()
pin_1.reset_index(inplace=True)
pin_1=pin_1.drop(["index"],axis=1)
pin_1.rename(columns = {'Order ID':'Order Id'}, inplace = True)

merge_3=pd.merge(df,pin_1,on="Order Id")
merge_3["Zone"]=merge_3["Zone"].str.upper()

df=merge_3.drop(['Warehouse Pincode','Customer Pincode'],axis=1)
df["Delivery Zone charged by Courier Company"]=invoice["Zone"].str.upper()

price=pd.DataFrame()
price['Type'] = invoice['Type of Shipment'].apply(lambda x: '1' if x == "Forward charges" else '2')
price["Order ID"]=invoice["Order ID"]
price['AWB Code']=invoice["AWB Code"]
price["Customer Pincode"]=invoice['Customer Pincode']
price["company zone"]=invoice["Zone"]
price["original zone"]=pin_code["Zone"]
price["weight"]=df["Weight slab as per X (KG)"]

def rate(Type, original_zone, weight):
    if Type == '1':
        if original_zone == "b":
            if weight == 0.5:
                return 33
            else:
                return ((((weight)/0.5)-1)*28.3)+33
        elif original_zone == "d":
            if weight == 0.5:
                return 45.4
            else:
                return ((((weight)/0.5)-1)*44.8)+45.4
        elif original_zone == "e":
            if weight == 0.5:
                return 56.6
            else:
                return ((((weight)/0.5)-1)*55.5)+56.6

    elif Type == '2':
        if original_zone == "b":
            if weight == 0.5:
                return 20.5+33
            else:
                return ((((weight)/0.5)-1)*20.5)+20.5+((((weight)/0.5)-1)*28.3)+33
        elif original_zone == "d":
            if weight == 0.5:
                return 41.3+45.4
            else:
                return ((((weight)/0.5)-1)*44.8)+41.3+((((weight)/0.5)-1)*48.8)+45.4
        elif original_zone == "e":
            if weight == 0.5:
                return 50.7+56.6
            else:
                return ((((weight)/0.5)-1)*55.5)+50.7+((((weight)/0.5)-1)*55.5)+56.6

df['Expected Charge as per X (Rs.)'] = price.apply(lambda row: rate(row['Type'], row['original zone'], row['weight']), axis=1)


invoice.rename(columns={"Order ID":"Order Id"},inplace=True)
invoice.rename(columns={"Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.) "},inplace=True)
bill=invoice.drop(columns=["AWB Code","Charged Weight","Warehouse Pincode","Customer Pincode","Type of Shipment","Zone"],axis=1)
df=pd.merge(df,bill,on="Order Id")


df["Difference Between Expected Charges and Billed Charges (Rs.)"]=df["Expected Charge as per X (Rs.)"]-df["Charges Billed by Courier Company (Rs.) "]

df1=df
df['correct values'] =(df['Expected Charge as per X (Rs.)'] == df['Charges Billed by Courier Company (Rs.) ']).astype(int)
df['undercharged'] = (df['Charges Billed by Courier Company (Rs.) '] - df['Expected Charge as per X (Rs.)']).where(df['Expected Charge as per X (Rs.)'] < df['Charges Billed by Courier Company (Rs.) '], 0)
df['overcharged']=(df['Expected Charge as per X (Rs.)'] - df['Charges Billed by Courier Company (Rs.) ']).where(df['Expected Charge as per X (Rs.)'] > df['Charges Billed by Courier Company (Rs.) '], 0)
num_equal = (df['Expected Charge as per X (Rs.)'] == df['Charges Billed by Courier Company (Rs.) ']).sum()
less_than=(df['Expected Charge as per X (Rs.)'] < df['Charges Billed by Courier Company (Rs.) ']).sum()
greater_than=(df['Expected Charge as per X (Rs.)'] > df['Charges Billed by Courier Company (Rs.) ']).sum()

summary_table=pd.DataFrame(columns=["summary","Count","Amount(Rs.)"])
corrected = pd.DataFrame({'summary': 'Total orders where X has been correctly charged', 
                        'Count': num_equal , 
                        'Amount(Rs.)': df["Charges Billed by Courier Company (Rs.) "].sum()}, index=[0])
greaterthan= pd.DataFrame({'summary': 'Total Orders where X has been overcharged', 
                        'Count': greater_than , 
                        'Amount(Rs.)': df["overcharged"].sum()}, index=[1])
lessthan = pd.DataFrame({'summary': 'Total Orders where X has been undercharged', 
                        'Count': less_than, 
                        'Amount(Rs.)': df["undercharged"].sum()}, index=[2])
summary_table = pd.concat([summary_table, corrected,greaterthan,lessthan], ignore_index=True)

df.to_csv("Final_Output.csv")
summary_table.to_csv("summary_table.csv")

fatal: destination path 'cointab' already exists and is not an empty directory.


In [4]:
df.to_csv("Final_Output.csv")
summary_table.to_csv("summary_table.csv")